# The Battle of the Neighborhoods - Part 2

#### In the previous week, I talked about obesity in Chicago-USA , it's perils and how to curtail it. One of the solution to obesity is ample of exercise. In this section, I'm going to decsribe the code I've used to potentially solve this problem by identifying areas in the city where one can construct gyms. I am using K-means machine learning algorithm. 

#### Lets start with installing and importing the relevant packages and libraries we require.

In [1]:
!pip install geopy
!pip install folium
!pip install geocoder
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import geocoder
from geopy.geocoders import Nominatim
import requests
import json
import folium
import requests
from bs4 import BeautifulSoup
import lxml.html as lh

     |████████████████████████████████| 94 kB 7.2 MB/s  eta 0:00:01
     |████████████████████████████████| 98 kB 10.7 MB/s eta 0:00:01


#### Using beautifulsoup, we scrape the wikipedia page containing neighborhoods of chicago data.

In [14]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Chicago').text
soup = BeautifulSoup(url,'lxml')

#### Lets build a dataframe containing the neighborhood data.

In [15]:
table = soup.find('table')
elements = table.find_all('td')
neighborhood = []
community = []
for i in range(0, len(elements), 2):
    neighborhood.append(elements[i].text.strip())
    community.append(elements[i+1].text.strip())
        
df= pd.DataFrame(data=[neighborhood, community]).transpose()
df.columns = ['Neighborhood', 'Community']
df.head(15)

,Neighborhood,Community
0,Albany Park,Albany Park
1,Altgeld Gardens,Riverdale
2,Andersonville,Edgewater
3,Archer Heights,Archer Heights
4,Armour Square,Armour Square
5,Ashburn,Ashburn
6,Ashburn Estates,Ashburn
7,Auburn Gresham,Auburn Gresham
8,Avalon Park,Avalon Park
9,Avondale,Avondale


#### Since we only need need the list of neighborhoods, we will drop the 'community' column.

In [16]:
df1=df[['Neighborhood']]
df1.head(15)

,Neighborhood
0,Albany Park
1,Altgeld Gardens
2,Andersonville
3,Archer Heights
4,Armour Square
5,Ashburn
6,Ashburn Estates
7,Auburn Gresham
8,Avalon Park
9,Avondale


In [17]:
df1.shape

(246, 1)

#### Now that we have the neighborhood list, we will use geocoder package to get the location data consisting of latitudes and longitudes of each neighborhood.

In [18]:
# define a function to get coordinates
def get_loc(neighborhood):
    # initialize your variable to None
    lat_long_loc = None
    # loop until you get the coordinates
    while(lat_long_loc is None):
        g = geocoder.arcgis('{}, Chicago, USA'.format(neighborhood))
        lat_long_loc = g.latlng
    return lat_long_loc
loc = [ get_loc(neighborhood) for neighborhood in df1["Neighborhood"].tolist() ]
loc

[[41.968290000000025, -87.72337999999996],
 [41.65441000000004, -87.60224999999997],
 [41.98046000000005, -87.66833999999994],
 [41.811540000000036, -87.72555999999997],
 [41.834580000000074, -87.63188999999994],
 [41.74785000000003, -87.70994999999994],
 [41.941680047633895, -88.19880743859594],
 [41.74319000000003, -87.65503999999999],
 [41.745070000000055, -87.58815999999996],
 [41.93925000000007, -87.71124999999995],
 [41.948461993723676, -87.72324781824939],
 [41.81045000000006, -87.66273999999999],
 [41.92746000000005, -87.77733999999998],
 [41.937657886629665, -87.83136491205029],
 [41.937657886629665, -87.83136491205029],
 [41.937657886629665, -87.83136491205029],
 [41.71563000000003, -87.67280999999997],
 [41.69589469625638, -87.64999223233774],
 [41.69589469625638, -87.64999223233774],
 [41.884250000000065, -87.63244999999995],
 [41.94014200000004, -87.64945399999993],
 [41.97974590525077, -87.6839099763566],
 [41.73339000000004, -87.65811999999994],
 [41.92735700000003, -87.

#### We will now merge our neighborhood data with location data.

In [19]:
df_loc = pd.DataFrame(loc, columns=['Latitude', 'Longitude'])
df1['Latitude'] = df_loc['Latitude']
df1['Longitude'] = df_loc['Longitude']
df1.head(15)

,Neighborhood,Latitude,Longitude
0,Albany Park,41.968290,-87.723380
1,Altgeld Gardens,41.654410,-87.602250
2,Andersonville,41.980460,-87.668340
3,Archer Heights,41.811540,-87.725560
4,Armour Square,41.834580,-87.631890
5,Ashburn,41.747850,-87.709950
6,Ashburn Estates,41.941680,-88.198807
7,Auburn Gresham,41.743190,-87.655040
8,Avalon Park,41.745070,-87.588160
9,Avondale,41.939250,-87.711250


#### Lets get the geographical co-ordinates of Chicago.

In [20]:
address = 'Chicago, USA'
geolocator = Nominatim(user_agent="smy-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chicago, USA: {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chicago, USA: 41.8755616, -87.6244212.


#### Creating a map of Chicago using latitude and longitudinal values

In [21]:
map_c = folium.Map(location=[latitude, longitude], zoom_start=4)

# add markers to map
for lat, lng, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_c)

map_c

#### From the map, we notice that some of these neighborhoods are not located within Chicago City. Let's drop these places from our dataframe and generate a new map.

In [22]:
df1= df1[df1.Neighborhood != "Ashburn Estates"].reset_index(drop=True)
df1= df1[df1.Neighborhood != "Polish Downtown"].reset_index(drop=True)
df1= df1[df1.Neighborhood != "Ranch Triangle"].reset_index(drop=True)
df1= df1[df1.Neighborhood != "Lilydale"].reset_index(drop=True)
df1= df1[df1.Neighborhood != "Budlong Woods"].reset_index(drop=True)
df1= df1[df1.Neighborhood != "Legends South (Robert Taylor Homes)"].reset_index(drop=True)
df1= df1[df1.Neighborhood != "Talley's Corner"].reset_index(drop=True)
df1= df1[df1.Neighborhood != "Lakewood / Balmoral"].reset_index(drop=True)
df1= df1[df1.Neighborhood != "Sleepy Hollow"].reset_index(drop=True)
df1= df1[df1.Neighborhood != "West Chesterfield"].reset_index(drop=True)
df1= df1[df1.Neighborhood != "Fulton River District"].reset_index(drop=True)
df1= df1[df1.Neighborhood != "K-Town"].reset_index(drop=True)
df1= df1[df1.Neighborhood != "Old Town Triangle"].reset_index(drop=True)
df1= df1[df1.Neighborhood != "Fernwood"].reset_index(drop=True)
df1= df1[df1.Neighborhood != "Golden Gate"].reset_index(drop=True)
df1= df1[df1.Neighborhood != "Irving Woods"].reset_index(drop=True)
df1= df1[df1.Neighborhood != "Chrysler Village"].reset_index(drop=True)


In [23]:
map_c = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_c)

map_c

#### Now that we have the correct neighborhoods, lets use Foursquare to find venues within our neighborhoods.

In [24]:
# define Foursquare Credentials and Version
CLIENT_ID = 'EJXBC4Z3WZ2U5AJVUMW0FFSE0SGUX01O2N1BK4LSE2EKOYTH' # your Foursquare ID
CLIENT_SECRET = '4O4OPJ0FNSWWVWYAP0QQZR3KGY1JT0RUN2JAHUDRRVPQZGUE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EJXBC4Z3WZ2U5AJVUMW0FFSE0SGUX01O2N1BK4LSE2EKOYTH
CLIENT_SECRET:4O4OPJ0FNSWWVWYAP0QQZR3KGY1JT0RUN2JAHUDRRVPQZGUE


In [25]:
LIMIT = 200

def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

chicago_venues = getNearbyVenues(names=df1['Neighborhood'],
                                   latitudes=df1['Latitude'],
                                   longitudes=df1['Longitude']
                                  )
print(chicago_venues.shape)
chicago_venues.head(15)

(13526, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Albany Park,41.96829,-87.72338,Lawrence Fish Market,41.968280,-87.726250,Seafood Restaurant
1,Albany Park,41.96829,-87.72338,Chicago Kalbi Korean BBQ,41.968314,-87.722771,Korean Restaurant
2,Albany Park,41.96829,-87.72338,Starbucks,41.968911,-87.728817,Coffee Shop
3,Albany Park,41.96829,-87.72338,Hiromi's Oriental Restaurant,41.968144,-87.718719,Karaoke Bar
4,Albany Park,41.96829,-87.72338,Eugene Field Park,41.974274,-87.722146,Park
5,Albany Park,41.96829,-87.72338,Nighthawk,41.967974,-87.713415,Cocktail Bar
6,Albany Park,41.96829,-87.72338,Rojo Gusano,41.968425,-87.724549,Taco Place
7,Albany Park,41.96829,-87.72338,Ssyal Korean Restaurant and Ginseng House,41.968172,-87.733207,Korean Restaurant
8,Albany Park,41.96829,-87.72338,Peking Mandarin Resturant,41.968292,-87.715783,Chinese Restaurant
9,Albany Park,41.96829,-87.72338,Marie's Pizza & Liquors,41.968132,-87.731533,Pizza Place


#### Counting the number of venues for each neighborhood.

In [26]:
chicago_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Albany Park,71,71,71,71,71,71
Altgeld Gardens,6,6,6,6,6,6
Andersonville,100,100,100,100,100,100
Archer Heights,37,37,37,37,37,37
Armour Square,54,54,54,54,54,54
...,...,...,...,...,...,...
Wildwood,21,21,21,21,21,21
Woodlawn,30,30,30,30,30,30
Wrightwood,100,100,100,100,100,100


#### Lets find the unique venue categories.

In [27]:
print('There are {} uniques categories.'.format(len(chicago_venues['Venue Category'].unique())))

There are 414 uniques categories.


In [28]:
# one hot encoding
chicago_onehot = pd.get_dummies(chicago_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
chicago_onehot['Neighborhoods'] = chicago_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [chicago_onehot.columns[-1]] + list(chicago_onehot.columns[:-1])
chicago_onehot = chicago_onehot[fixed_columns]

chicago_onehot.head(10)

,Neighborhoods,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,...,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Albany Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Albany Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Albany Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Albany Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Albany Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Albany Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Albany Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Albany Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Albany Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Albany Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Grouping all the venues together.

In [29]:
chicago_grouped = chicago_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(chicago_grouped.shape)
chicago_grouped

(229, 415)


,Neighborhoods,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,...,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Albany Park,0.0,0.0,0.00,0.014085,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.0,0.014085,0.0,0.00,0.0,0.0
1,Altgeld Gardens,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.0,0.000000,0.0,0.00,0.0,0.0
2,Andersonville,0.0,0.0,0.01,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.01,0.00,0.01,0.0,0.000000,0.0,0.01,0.0,0.0
3,Archer Heights,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.0,0.027027,0.0,0.00,0.0,0.0
4,Armour Square,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.0,0.000000,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,Wildwood,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.0,0.000000,0.0,0.00,0.0,0.0
225,Woodlawn,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.0,0.000000,0.0,0.00,0.0,0.0
226,Wrightwood,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.02,0.01,0.00,0.0,0.000000,0.0,0.03,0.0,0.0
227,Wrightwood Neighbors,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.02,0.01,0.00,0.0,0.000000,0.0,0.03,0.0,0.0


#### Lets find all the unique venue categories.

In [30]:
chicago_venues['Venue Category'].unique()

array(['Seafood Restaurant', 'Korean Restaurant', 'Coffee Shop',
       'Karaoke Bar', 'Park', 'Cocktail Bar', 'Taco Place',
       'Chinese Restaurant', 'Pizza Place', 'Mexican Restaurant',
       'Fried Chicken Joint', 'Discount Store', 'Dive Bar',
       'Mobile Phone Shop', 'Fish Market', 'Latin American Restaurant',
       'Grocery Store', 'Bakery', 'Pet Store', 'Afghan Restaurant',
       'Donut Shop', 'Sandwich Place', 'Hot Dog Joint', 'Thai Restaurant',
       'Ice Cream Shop', 'Wings Joint', 'Pharmacy', 'Cosmetics Shop',
       'Fast Food Restaurant', 'Sushi Restaurant', 'Convenience Store',
       'Supermarket', 'Bank', 'Hookah Bar', 'Asian Restaurant',
       'Thrift / Vintage Store', 'Pub', 'Train Station', 'Gas Station',
       'Bus Station', 'Pool Hall', 'Gaming Cafe', 'Automotive Shop',
       'BBQ Joint', 'Food', 'Harbor / Marina', 'Clothing Store',
       'Pie Shop', 'Italian Restaurant', 'Café', 'Breakfast Spot',
       'Greek Restaurant', 'New American Restaurant', '

#### The categories we're interested in are Gyms, Gym / Fitness Center and Yoga Studio. Let's check how many of these exist on our map.

In [31]:
print(len(chicago_grouped[chicago_grouped["Gym / Fitness Center"] > 0]))
print(len(chicago_grouped[chicago_grouped["Gym"] > 0]))
print(len(chicago_grouped[chicago_grouped["Yoga Studio"] > 0]))

86
92
51


#### Let's build a new dataframe refecting these categories.

In [32]:
chicago_gym = chicago_grouped[["Neighborhoods","Gym","Gym / Fitness Center", "Yoga Studio"]]
chicago_gym.head()

,Neighborhoods,Gym,Gym / Fitness Center,Yoga Studio
0,Albany Park,0.000000,0.000000,0.00
1,Altgeld Gardens,0.000000,0.000000,0.00
2,Andersonville,0.020000,0.020000,0.01
3,Archer Heights,0.000000,0.027027,0.00
4,Armour Square,0.018519,0.000000,0.00


### K-Means Clustering

In [34]:
# set number of clusters
kclusters = 10

chicago_clustering = chicago_gym.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(chicago_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 7, 0, 2, 1, 1, 1, 3, 1], dtype=int32)

In [35]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
chicago_merged = chicago_gym.copy()

# add clustering labels
chicago_merged["Cluster Labels"] = kmeans.labels_

chicago_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
chicago_merged.head()

,Neighborhood,Gym,Gym / Fitness Center,Yoga Studio,Cluster Labels
0,Albany Park,0.000000,0.000000,0.00,1
1,Altgeld Gardens,0.000000,0.000000,0.00,1
2,Andersonville,0.020000,0.020000,0.01,7
3,Archer Heights,0.000000,0.027027,0.00,0
4,Armour Square,0.018519,0.000000,0.00,2


#### Merge Chicago_grouped with Chicago_data to add latitude/longitude for each neighborhood

In [36]:
chicago_merged = chicago_merged.join(df1.set_index("Neighborhood"), on="Neighborhood")

print(chicago_merged.shape)
chicago_merged.head()

(229, 7)


,Neighborhood,Gym,Gym / Fitness Center,Yoga Studio,Cluster Labels,Latitude,Longitude
0,Albany Park,0.000000,0.000000,0.00,1,41.96829,-87.72338
1,Altgeld Gardens,0.000000,0.000000,0.00,1,41.65441,-87.60225
2,Andersonville,0.020000,0.020000,0.01,7,41.98046,-87.66834
3,Archer Heights,0.000000,0.027027,0.00,0,41.81154,-87.72556
4,Armour Square,0.018519,0.000000,0.00,2,41.83458,-87.63189


#### Sorting the results by Cluster Labels

In [37]:
print(chicago_merged.shape)
chicago_merged.sort_values(["Cluster Labels"], inplace=True)
chicago_merged

(229, 7)


,Neighborhood,Gym,Gym / Fitness Center,Yoga Studio,Cluster Labels,Latitude,Longitude
100,LeClaire Courts,0.00,0.028571,0.00,0,41.907715,-87.753430
113,Marshall Square,0.00,0.032787,0.00,0,41.852026,-87.699066
66,Garfield Ridge,0.00,0.050000,0.00,0,41.790760,-87.769850
120,Mount Greenwood,0.00,0.030303,0.00,0,41.691450,-87.708300
156,Princeton Park,0.00,0.040000,0.00,0,41.745680,-87.631710
...,...,...,...,...,...,...,...
70,Graceland West,0.01,0.020000,0.02,9,41.957229,-87.662274
68,Gold Coast,0.03,0.020000,0.02,9,41.903950,-87.628860
226,Wrightwood,0.03,0.020000,0.03,9,41.928979,-87.656190
168,Roscoe Village,0.02,0.030000,0.03,9,41.944590,-87.678490


#### Visualizing.

In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(chicago_merged['Latitude'], chicago_merged['Longitude'], chicago_merged['Neighborhood'], chicago_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Finally, let's analyze each cluster

#### Cluster 0

In [39]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 0]

,Neighborhood,Gym,Gym / Fitness Center,Yoga Studio,Cluster Labels,Latitude,Longitude
100,LeClaire Courts,0.000000,0.028571,0.00,0,41.907715,-87.753430
113,Marshall Square,0.000000,0.032787,0.00,0,41.852026,-87.699066
66,Garfield Ridge,0.000000,0.050000,0.00,0,41.790760,-87.769850
120,Mount Greenwood,0.000000,0.030303,0.00,0,41.691450,-87.708300
156,Princeton Park,0.000000,0.040000,0.00,0,41.745680,-87.631710
30,Calumet Heights,0.000000,0.025641,0.00,0,41.733360,-87.577420
144,Oriole Park,0.000000,0.026316,0.00,0,41.979030,-87.814430
191,Stony Island Park,0.000000,0.030303,0.00,0,41.735310,-87.574490
181,South Austin,0.000000,0.034483,0.00,0,41.788354,-87.771916
117,Merchant Park,0.000000,0.025641,0.00,0,41.927816,-87.706418


#### Cluster 1

In [40]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 1]

,Neighborhood,Gym,Gym / Fitness Center,Yoga Studio,Cluster Labels,Latitude,Longitude
134,North Park,0.0,0.0,0.0,1,41.98294,-87.71915
0,Albany Park,0.0,0.0,0.0,1,41.96829,-87.72338
140,Old Edgebrook,0.0,0.0,0.0,1,41.99732,-87.76423
139,Oakland,0.0,0.0,0.0,1,41.82496,-87.60565
146,Park Manor,0.0,0.0,0.0,1,41.77039,-87.61859
...,...,...,...,...,...,...,...
72,Grand Crossing,0.0,0.0,0.0,1,41.75671,-87.59925
81,Hermosa,0.0,0.0,0.0,1,41.92848,-87.73424
71,Grand Boulevard,0.0,0.0,0.0,1,41.80931,-87.62142
1,Altgeld Gardens,0.0,0.0,0.0,1,41.65441,-87.60225


#### Cluster 2

In [41]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 2]

,Neighborhood,Gym,Gym / Fitness Center,Yoga Studio,Cluster Labels,Latitude,Longitude
4,Armour Square,0.018519,0.00,0.000000,2,41.834580,-87.631890
148,Parkview,0.012048,0.00,0.000000,2,41.950895,-87.724646
18,Big Oaks,0.020000,0.00,0.000000,2,41.884250,-87.632450
169,Rosehill,0.015152,0.00,0.015152,2,41.986691,-87.672655
167,Rogers Park,0.018519,0.00,0.000000,2,42.008970,-87.666190
11,Belmont Central,0.011905,0.00,0.000000,2,41.927460,-87.777340
172,Saint Ben's,0.020000,0.00,0.000000,2,41.884250,-87.632450
209,West DePaul,0.020000,0.00,0.000000,2,41.879730,-87.633140
203,Wacławowo,0.020000,0.00,0.000000,2,41.884250,-87.632450
201,Uptown,0.023256,0.00,0.011628,2,41.965380,-87.669360


#### Cluster 3

In [42]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 3]

,Neighborhood,Gym,Gym / Fitness Center,Yoga Studio,Cluster Labels,Latitude,Longitude
162,Ravenswood Manor,0.013699,0.013699,0.000000,3,41.994385,-87.675412
163,River North,0.000000,0.020000,0.010000,3,41.892520,-87.624770
124,New Chinatown,0.010000,0.010000,0.000000,3,41.852770,-87.634990
161,Ravenswood Gardens,0.013699,0.013699,0.000000,3,41.994385,-87.675412
96,Kosciuszko Park,0.012346,0.012346,0.000000,3,41.930040,-87.724530
61,Ford City,0.000000,0.013514,0.000000,3,41.753151,-87.732240
32,Central Station,0.000000,0.023256,0.000000,3,41.790714,-87.762192
15,Beverly,0.000000,0.014286,0.000000,3,41.715630,-87.672810
118,Montclare,0.013158,0.013158,0.000000,3,41.929020,-87.798470
109,Loyola,0.010000,0.020000,0.000000,3,42.001549,-87.658629


#### Cluster 4

In [43]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 4]

,Neighborhood,Gym,Gym / Fitness Center,Yoga Studio,Cluster Labels,Latitude,Longitude
69,Goose Island,0.060000,0.01,0.020000,4,41.903346,-87.653003
173,Sauganash,0.041667,0.00,0.041667,4,41.989970,-87.742270
122,Near North Side,0.050000,0.04,0.030000,4,41.900210,-87.634330
143,Old Town,0.080000,0.04,0.010000,4,41.905390,-87.641790


#### Cluster 5

In [44]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 5]

,Neighborhood,Gym,Gym / Fitness Center,Yoga Studio,Cluster Labels,Latitude,Longitude
42,Dearborn Homes,0.0,0.105263,0.0,5,41.84309,-87.62783


#### Cluster 6

In [45]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 6]

,Neighborhood,Gym,Gym / Fitness Center,Yoga Studio,Cluster Labels,Latitude,Longitude
14,Belmont Terrace,0.00,0.00,0.027027,6,41.937658,-87.831365
107,Longwood Manor,0.00,0.00,0.023256,6,41.699174,-87.671217
85,Hyde Park,0.01,0.01,0.020000,6,41.793880,-87.593260
106,Logan Square,0.01,0.01,0.020000,6,41.923280,-87.698100
200,University Village,0.00,0.00,0.018868,6,41.863540,-87.647390
149,Peterson Park,0.00,0.00,0.055556,6,41.985820,-87.728480
216,West Loop,0.00,0.02,0.030000,6,41.883040,-87.653350
97,Lake Meadows,0.00,0.01,0.020000,6,41.959896,-87.659426
12,Belmont Gardens,0.00,0.00,0.027027,6,41.937658,-87.831365
13,Belmont Heights,0.00,0.00,0.027027,6,41.937658,-87.831365


#### Cluster 7

In [46]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 7]

,Neighborhood,Gym,Gym / Fitness Center,Yoga Studio,Cluster Labels,Latitude,Longitude
74,Greektown,0.020000,0.020000,0.01,7,41.877723,-87.646865
2,Andersonville,0.020000,0.020000,0.01,7,41.980460,-87.668340
20,Boystown,0.020000,0.010000,0.01,7,41.940142,-87.649454
136,Norwood Park East,0.030000,0.030000,0.01,7,41.991749,-87.665239
228,Wrigleyville,0.030000,0.020000,0.00,7,41.947250,-87.653200
53,Eden Green,0.025000,0.025000,0.00,7,42.011718,-87.699950
52,East Village,0.020000,0.010000,0.01,7,41.899640,-87.672430
135,Nortown,0.030000,0.030000,0.00,7,41.910475,-87.660623
142,Old Norwood,0.030000,0.030000,0.01,7,41.991749,-87.665239
132,North Lawndale,0.016667,0.016667,0.00,7,41.909031,-87.719224


#### Cluster 8

In [47]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 8]

,Neighborhood,Gym,Gym / Fitness Center,Yoga Studio,Cluster Labels,Latitude,Longitude
24,Brighton Park,0.035714,0.0,0.0,8,41.818610,-87.699480
77,Hamilton Park,0.050000,0.0,0.0,8,41.763460,-87.637440
164,River West,0.037037,0.0,0.0,8,41.948815,-87.855969
171,Rosemoor,0.040404,0.0,0.0,8,41.973515,-87.865460
119,Morgan Park,0.041667,0.0,0.0,8,41.690390,-87.665990
63,Fuller Park,0.040000,0.0,0.0,8,41.812530,-87.632620
28,Burnside,0.052632,0.0,0.0,8,41.729440,-87.597680


#### Cluster 9

In [48]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 9]

,Neighborhood,Gym,Gym / Fitness Center,Yoga Studio,Cluster Labels,Latitude,Longitude
157,Printer's Row,0.03,0.02,0.02,9,41.927074,-87.651288
129,North Center,0.02,0.01,0.02,9,41.954110,-87.681420
177,Sheffield Neighbors,0.02,0.03,0.02,9,41.921735,-87.653490
221,West Town,0.01,0.03,0.02,9,41.893290,-87.657430
27,Buena Park,0.01,0.02,0.02,9,41.958446,-87.652636
26,Bucktown,0.02,0.01,0.02,9,41.917850,-87.672870
70,Graceland West,0.01,0.02,0.02,9,41.957229,-87.662274
68,Gold Coast,0.03,0.02,0.02,9,41.903950,-87.628860
226,Wrightwood,0.03,0.02,0.03,9,41.928979,-87.656190
168,Roscoe Village,0.02,0.03,0.03,9,41.944590,-87.678490


## Thank You for reading!!!!